In [1]:
from abides_core import abides
from abides_core.agent import Agent
from abides_core.communicative_agent import CommunicativeAgent
from abides_core.network import *
from abides_core.message import Message

In [2]:
from itertools import count

id_gen = count(0)

# Sieci kontaktów [moduł *network*]

In [3]:
class ExampleAgent(CommunicativeAgent):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.received_messages = []

    def wakeup(self, current_time):
        super().wakeup(current_time)
        self.broadcast(Message())

    def receive_message(self, current_time, sender_id, message):
        super().receive_message(current_time, sender_id, message)
        self.received_messages.append((current_time, sender_id, message))


In [4]:
import logging

logger = logging.getLogger("abides")


class ExampleAgent2(CommunicativeAgent):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.received_messages = []

    def wakeup(self, current_time):
        super().wakeup(current_time)
        self.broadcast(Message())

    def receive_message(self, current_time, sender_id, message):
        #logger.info(f"{type(message)}, {sender_id}, {current_time}")
        super().receive_message(current_time, sender_id, message)
        self.received_messages.append((current_time, sender_id, message))
        self.broadcast(message)

In [5]:
#graph = CompleteGraph.construct(agent_types={ExampleAgent: 5}, id_generator=id_gen)

In [6]:
graph_0 = CentralizedNetwork.construct(central_agent_type=ExampleAgent2, agent_types={ExampleAgent: 100},
                                       id_generator=id_gen)

In [8]:
from abides_markets.configs.utils import generate_BA_graph_sequence

In [9]:
seq_gen = generate_BA_graph_sequence(graph_0, 4, [ExampleAgent(id=next(id_gen)) for i in range(1000)], [105,115,125,200,500,1000])

In [10]:
g_1 = next(seq_gen)

In [15]:
g_1.agents[104].contacts

[9, 0, 82, 13]

In [8]:
graph = BANetwork.construct(starting_state=graph_0, agent_types={ExampleAgent: 1000}, m=10,
                                   id_generator=id_gen)

In [6]:
graph= ERNetwork.construct(agent_types={ExampleAgent: 1000}, p=0.99,
                            id_generator=id_gen)

In [9]:
graph.agents

 ...]

In [10]:
len([agent for agent in iter(graph.agents) if agent.contacts is not None])

1101

In [11]:
len(graph.agents[0].contacts)

996

In [12]:
len(graph.agents[999].contacts)

12

In [ ]:
end_state = abides.run(config={"agents": graph.get_agent_list(), "stdout_log_level": "INFO"})

In [ ]:
end_state

In [ ]:
end_state['agents'][2].received_messages

In [ ]:
end_state['agents'][0].received_messages

In [ ]:
graph.agents[0].contacts

# Message Processing Models

In [ ]:
from abides_markets.messages.trading_signal import SellSignal, BuySignal
from abides_core.message_processing_model import MostRecentProcessingModel, VotingProcessingModel


class ExampleAgent3(CommunicativeAgent):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.received_messages = []

    def wakeup(self, current_time):
        super().wakeup(current_time)
        self.broadcast(BuySignal(symbol="test"))


class ExampleAgent4(CommunicativeAgent):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.received_messages = []

    def wakeup(self, current_time):
        super().wakeup(current_time)
        self.broadcast(SellSignal(symbol="test"))


class ExampleAgent5(CommunicativeAgent):
    def __init__(self, processing_model, **kwargs):
        super().__init__(**kwargs)
        self.received_messages = []
        self.message_processing_model = processing_model
        self.final_decision = None

    def wakeup(self, current_time):
        super().wakeup(current_time)
        self.broadcast(Message())

    def receive_message(self, current_time, sender_id, message):
        super().receive_message(current_time, sender_id, message)
        self.message_processing_model.on_receive(current_time, message, sender_id)
        self.received_messages.append((current_time, sender_id, message))

    def kernel_terminating(self):
        self.final_decision = type(self.message_processing_model.on_decision())
        super().kernel_terminating()


In [ ]:
# test 0: dwa centra jedno most recent, jedno voting
# test probabilistic: dwa centra jedno TrustWeighted, jedno TrustedSource, agenci wysyłający sygnał raz na x nanosekund

In [ ]:
agents_graph_1 = [ExampleAgent4(id=next(id_gen)) for i in range(4)]
agents_graph_1.extend([ExampleAgent3(id=next(id_gen)) for i in range(6)])

agents_graph_2 = [ExampleAgent3(id=next(id_gen)) for i in range(5)]
agents_graph_2.extend([ExampleAgent4(id=next(id_gen)) for i in range(5)])

central_agent_1 = ExampleAgent5(processing_model=VotingProcessingModel(), id=next(id_gen))
central_agent_2 = ExampleAgent5(processing_model=MostRecentProcessingModel(), id=next(id_gen))

graph_1 = CentralizedNetwork.construct_from_agent_list(central_agent=central_agent_1, agent_list=agents_graph_1)
graph_2 = CentralizedNetwork.construct_from_agent_list(central_agent=central_agent_2, agent_list=agents_graph_2)

In [ ]:
central_agent_1.id

In [ ]:
sim_agents = graph_1.get_agent_list()
sim_agents.extend(graph_2.get_agent_list())
sim_agents.sort(key=lambda agent: agent.id)

In [ ]:
[sim_agent.id for sim_agent in iter(sim_agents)]

In [ ]:
end_state = abides.run(config={"agents": sim_agents, "stdout_log_level": "INFO"})

In [ ]:
end_state['agents']

In [ ]:
# czemu nonetype? czy dostaje właściwą wiadomosc?
end_state['agents'][20].received_messages

In [ ]:
end_state['agents'][21].final_decision

In [ ]:
end_state['agents'][20].final_decision

# *Friend*, *FalseFriend*, *Insider*

In [ ]:
from abides_markets.agents.network_agents import Friend, FalseFriend

graph_1.get_agent_list()